In [1]:
%matplotlib inline

import pandas as pd
from matplotlib import pyplot as plt

# Enunciado

2)  (\*\*\*\*) (15 pts) El GCPD (Gotham City Police Dep
t) recolecta la información de casos policiales que
 acontecen en 
Ciudad Gótica. Esta información se encuentra guarda
da en un dataframe con el siguiente formato: 
(fecha, id_caso, 
descripcion, estado_caso, categoria, latitud, longi
tud).  
Los  posibles estados  que puede tener  un caso son 1:
 caso abierto, 2: caso  resuelto, 3: cerrado sin res
olución.  Las 
fechas se encuentran en el formato YYYY-MM-DD. 
Por otro lado el comisionado Gordon guarda un regis
tro detallado sobre en cuáles casos fue activada la
 batiseñal para 
pedir ayuda del vigilante, Batman. Esta información
 se encuentra en un Dataframe con el siguiente formato
 (id_caso, respuesta)
siendo campo respuesta si la señal tuvo una respu
esta positiva (1) o negativa (0) de parte de él 
El  sector  encargado  de  las  estadísticas  oficiales  d
el  GCPD  quiere  con  esta  información
analizar  las  siguientes situaciones: 
- Tasa de resolución de casos de la fuerza policial
 por categoría de caso (considerando aquellos casos
 en los que no 
participó Batman). 
- Tasa de resolución de casos con la ayuda de Batma
n (considerando que aquellos casos en los que fue l
lamado con la batiseñal, participó en la resolución).
- Indicar el mes del año pasado en el que Batman tuvo
mayor participación en la investigación de casos.

# Creación de los datos

In [32]:
d = {'fecha': ['2018-04-23', '2018-05-20', '2018-05-20'], 'id_caso': [1, 2, 3], 'descripcion': ['a', 'b', 'c'], 'estado_caso': [2, 2, 2], 'categoria': [1, 2, 3], 'latitud': [1, 2, 3], 'longitud': [1, 2, 3]}
casos = pd.DataFrame(d)
casos

,categoria,descripcion,estado_caso,fecha,id_caso,latitud,longitud
0,1,a,2,2018-04-23,1,1,1
1,2,b,2,2018-05-20,2,2,2
2,3,c,2,2018-05-20,3,3,3


In [25]:
d = {'id_caso': [1, 2], 'respuesta': [1, 0]}
casos_batman = pd.DataFrame(d)
casos_batman

,id_caso,respuesta
0,1,1
1,2,0


# Tasa de resolución de casos por categoría de caso

In [26]:
resueltos = casos.loc[casos.estado_caso == 2]
resueltos

,categoria,descripcion,estado_caso,fecha,id_caso,latitud,longitud
0,1,a,2,2018-04-23,1,1,1
1,2,b,2,2018-05-20,2,2,2
2,3,c,2,2018-05-20,2,3,3


In [27]:
tasa_resueltos_por_categoria = resueltos['categoria'].value_counts(normalize = True)
tasa_resueltos_por_categoria

3    0.333333
2    0.333333
1    0.333333
Name: categoria, dtype: float64

In [28]:
tasas = tasa_resueltos_por_categoria.values
tasas

array([ 0.33333333,  0.33333333,  0.33333333])

In [29]:
categorias = tasa_resueltos_por_categoria.index
categorias

Int64Index([3, 2, 1], dtype='int64')

In [30]:
for i in range(0, len(categorias)):
    print('Categoría: ' + str(categorias[i]) + ' Tasa: ' + str(tasas[i]))

Categoría: 3 Tasa: 0.333333333333
Categoría: 2 Tasa: 0.333333333333
Categoría: 1 Tasa: 0.333333333333


# Tasa de resolución de casos con ayuda de Batman

### Creamos columna de ayuda de batman en el df de casos

In [38]:
# Hacemos join con los casos
casos_join = pd.DataFrame()
# Tener en cuenta que el join te settea a how='left', y como en el left tenemos el df de casos que tiene
# todos los casos totales, no perdemos información.
# Si agarrabamos con right (casos_batman) perdemos todos aquellos casos que no están es este df
# Osea, que perdemos todos los casos en donde no se llamo a la batiseñal.
# Se puede hacer con merge también, pero me es más lindo así. :)
casos_join = casos.join(casos_batman.set_index('id_caso'), on = 'id_caso')
casos_join

,categoria,descripcion,estado_caso,fecha,id_caso,latitud,longitud,respuesta
0,1,a,2,2018-04-23,1,1,1,1.0
1,2,b,2,2018-05-20,2,2,2,0.0
2,3,c,2,2018-05-20,3,3,3,NaN


- Ver que nos queda un NaN, ese es porque es un caso que no estaba en el df de `casos_batman`, entonces
no se tenía la información de `respuesta`.
- Podemos hacer una columna en donde poner la información de si se llamo a la batiseñal o no. O
  tal vez una que nos diga directamente si participó o no. La segunda es más útil para lo que nos piden.
- Tener en cuenta que nos piden tres cosas diferentes. Si las tomamos como que las hacemos todas
  es más eficiente hacer primero el join y después ir filtrando por lo que nos piden. Si los tomamos
  por separado es más eficiente primero filtrar y después hacer el join.

In [16]:
# Saco id_casobatman
#casos_join = casos_join.drop('id_casobatman', axis=1)
#casos_join

### Calculo tasa de resolución de casos con ayuda de batman

In [39]:
tasas = casos_join.respuesta.value_counts(normalize = True)
tasas

0.0    0.5
1.0    0.5
Name: respuesta, dtype: float64

In [40]:
print('La tasa de resolución de casos con ayuda de batman es: ' + str(tasas[1]))

La tasa de resolución de casos con ayuda de batman es: 0.5


# Mes con más ayuda de Batman

In [19]:
# Paso a timestamp fecha
casos_join['fecha'] = pd.to_datetime(casos_join.fecha)

In [20]:
# Obtengo el mes
casos_join['mes'] = casos_join.fecha.dt.month
casos_join

,categoria,descripcion,estado_caso,fecha,latitud,longitud,respuesta,mes
id_caso,,,,,,,,
1,1,a,2,2018-04-23,1,1,1,4
2,2,b,2,2018-05-20,2,2,0,5


In [21]:
casos_join.loc[casos_join.respuesta == 1].mes.value_counts()

4    1
Name: mes, dtype: int64

Se puede hacer con groupby?

In [23]:
casos_join[]

0    1
1    2
Name: categoria, dtype: object
0    a
1    b
Name: descripcion, dtype: object
0    2
1    2
Name: estado_caso, dtype: object
0    2018-04-23
1    2018-05-20
Name: fecha, dtype: object
0    1
1    2
Name: id_caso, dtype: object
0    1
1    2
Name: latitud, dtype: object
0    1
1    2
Name: longitud, dtype: object
0    1
1    0
Name: respuesta, dtype: object


categoria      None
descripcion    None
estado_caso    None
fecha          None
id_caso        None
latitud        None
longitud       None
respuesta      None
dtype: object